![img](img/dark_library.png)

# **App de consulta de documentos mediante GPT en AWS**

Steven Noboa, María Neches, Manuel Reina Águeda González

## Introducción

## 1. Elaboración de un primer borrador del script y conexión a Langchain
+ Primera template html
+ Desarrollo del script con una primera base de datos local de prueba
+ Conexión a Langchain

```python

 fragment_size = 4096
        while True:
            parte = archivo.read(fragment_size)
            if not parte:
                break

            try:
                response = qa_document_chain.run(
                    input_document=parte.decode('latin-1'),
                    question=pregunta
                    )
```

## 3. Creación de la instancia en AWS - RDS y de la base de datos MySQL
+ Creamos una instancia de la base de datos en la plataforma de AWS y nos conectamos a ella (puerto 3306).
+ Una vez conectados, creamos también la base de datos definitiva (gpt_database) y la tabla correspondiente (gpt_table), que consta de las siguientes columnas:
```python
gpt_table (
Registro DATETIME,
Nombre TEXT,
Archivo TEXT,
Consulta TEXT,
Respuesta TEXT)
```

- *Registro*: almacena la fecha y hora de la consulta realizada;
- *Nombre*: nombre de la persona que realiza la consulta;
- *Archivo*: nombre y extensión del archivo subido para la consulta;
- *Consulta*: prompt enviado a gpt para la consulta;
- *Respuesta*: resultados de la consulta a gpt.

## 4. Desarrollo de la App
### Primer endpoint: 
+ La app lee archivos con las extensiones ```.py```, ```.ipynb``` y ```.txt```

+ Habilitamos dos opciones para la API Key de OpenAI.
    + De forma predeterminada, la key es una variable de entorno y no está disponible para el usuario, de forma que cuando este realiza una consulta la key se introduce automáticamente de forma invisible.
    
    + La otra opción disponible es un campo que el usuario puede completar con su propia API Key para realizar consultas.

```python
@app.route('/')
def home():
    return render_template('index.html')

@app.route('/analizar_documento', methods=['POST'])
def analizar_documento():
    openai_api_key = request.form.get("api_key")

    if openai_api_key:
        
        llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)
        qa_chain = load_qa_chain(llm, chain_type="map_reduce")
        qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain) 
    else:
        openai_api_key= openai_key_private
    
        llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)
        qa_chain = load_qa_chain(llm, chain_type="map_reduce")
        qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)
    
    if 'archivo' in request.files:
        archivo = request.files['archivo']
        if not archivo:
            return "Error: Ningún archivo introducido"
                
        nombre = request.form.get('nombre')
        if not nombre:
            return "Error: Debe introducir su nombre"
        
        pregunta = request.form.get('pregunta')
        if not pregunta:
            return "Error: La pregunta no se proporcionó"
        
        tipos_aceptados = {'.txt', '.py', '.ipynb'}

        if archivo.filename and not any(archivo.filename.lower().endswith(ext) for ext in tipos_aceptados):
            error_message = "Error: Tipo de archivo no admitido"
            return render_template('index.html', error_message=error_message)

        # leer el archivo en fragmentos de 4KB
                fecha_hora_actual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                insertar_registro(fecha_hora_actual, nombre,archivo.filename, pregunta, str(response))

                return render_template('respuestas.html', pregunta=pregunta, respuesta=str(response))
            except Exception as e:
                return render_template('respuestas.html', pregunta=pregunta, respuesta=str(e))
```
### Otros endpoints

+ Decidimos crear otros dos endpoints (```/consultas``` y ```/realizar_consulta```) que permiten también realizar consultas sobre los registros de la base de datos filtrándolos por nombre del usuario, aunque este se escriba en minúsculas o no esté completo

```python
@app.route('/consultas')
def consultas():
    return render_template('consultas.html')

@app.route('/realizar_consulta', methods=['GET'])
def realizar_consulta():
    nombre = request.args.get('nombre')
    conn = pymysql.connect(
        host=host,
        user=username,
        port=port,
        password=password,
        database=database
    )
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    try:
        if nombre:
            consulta = f"SELECT * FROM gpt_table WHERE nombre LIKE LOWER('%{nombre}%')"
        else:
            consulta = "SELECT * FROM gpt_table"

        cursor.execute(consulta)
        resultados = cursor.fetchall()
    except Exception as e:
        print(f"Error al ejecutar la consulta: {str(e)}")
        resultados = []

    finally:
        cursor.close()
        conn.close()
    return render_template('results_BBDD.html', resultados=resultados)```

+ Para cada uno de estos endpoints, creamos una interfaz en html


## 5. Test de la App
+ Desarrollamos los [test](../test/test.py) para poner la app a prueba y confirmar que nos devolvía código 200 en todos sus endpoints

![imagen](img/test_pasados.jpg)

+ Los test están diseñados para poder ejecutarlos sobre la imagen dockerizada, aunque no se disponga del archivo local.

## 6. Dockerización y despliegue de la App
+ Creamos los archivos de [Dockerfile](../app/dockerfile) y [requirements](../app/requirements.txt) para dockerizar la app
+ Creamos la imagen final en docker (manuelreina/app_gpt:v1) e hicimos un push para subirla a Docker Hub

Para ejecutarla, se deberá ejecutar el siguiente comando:

```docker run -p 5000:5000 manuelreina/app_gpt:v1```